In [7]:
!python -V
import warnings
warnings.filterwarnings("ignore")

Python 3.9.12


In [8]:
import pandas as pd

In [9]:
import pickle

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [12]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment1")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1715948834814, experiment_id='2', last_update_time=1715948834814, lifecycle_stage='active', name='nyc-taxi-experiment1', tags={}>

In [13]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [14]:
df_train = read_dataframe('/workspaces/mlops-zoomcamp/data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('/workspaces/mlops-zoomcamp/data/yellow_tripdata_2023-02.parquet')

In [15]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [16]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

In [17]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [18]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [19]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

5.198291544889195

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [21]:
del df_train
del df_val

In [18]:
with mlflow.start_run():

    mlflow.set_tag("Data Scientist", "Microsoft")

    mlflow.log_param("train-data-path", '/workspaces/mlops-zoomcamp/data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param("valid-data-path", '/workspaces/mlops-zoomcamp/data/yellow_tripdata_2023-02.parquet')

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [22]:
import xgboost as xgb

In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [24]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=4,
            evals=[(valid, 'validation')],
            early_stopping_rounds=3
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=4,
    trials=Trials()
)

[0]	validation-rmse:9.69556                                                                               
[1]	validation-rmse:9.34654                                                                               
[2]	validation-rmse:9.01953                                                                               
[3]	validation-rmse:8.71332                                                                               
[0]	validation-rmse:6.97125                                                                               
[1]	validation-rmse:5.74295                                                                               
[2]	validation-rmse:5.30380                                                                               
[3]	validation-rmse:5.14250                                                                               
[0]	validation-rmse:8.22865                                                                               
[1]	validation-rmse:7.00992          

In [24]:
mlflow.xgboost.autolog(disable=True)

In [20]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=4,
        evals=[(valid, 'validation')],
        early_stopping_rounds=2
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:9.37133
[1]	validation-rmse:8.75894
[2]	validation-rmse:8.22242
[3]	validation-rmse:7.75402


In [21]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (GradientBoostingRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", '/workspaces/mlops-zoomcamp/data/yellow_tripdata_2023-01.parquet')
        mlflow.log_param("valid-data-path", '/workspaces/mlops-zoomcamp/data/yellow_tripdata_2023-02.parquet')
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/05/20 08:29:42 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: Unable to allocate 489. GiB for an array with shape (3009173, 21802) and data type float64
2024/05/20 08:32:49 WARNING mlflow.sklearn: Failed to log evaluation dataset information to MLflow Tracking. Reason: Unable to allocate 464. GiB for an array with shape (2855951, 21802) and data type float64
2024/05/20 08:32:49 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: Unable to allocate 489. GiB for an array with shape (3009173, 21802) and data type float64
2024/05/20 08:38:05 WARNING mlflow.sklearn: Failed to log evaluation dataset information to MLflow Tracking. Reason: Unable to allocate 464. GiB for an array with shape (2855951, 21802) and data type float64


In [2]:
import mlflow
path = "/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/0338e611743c4663b78fece083312bab/artifacts/model"
logged_model = path

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd


In [3]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 0338e611743c4663b78fece083312bab

In [5]:

new_model = "/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/47f6b87d87db416592332e2664952a59/artifacts/models_mlflow"
xgboost_model = mlflow.xgboost.load_model(new_model)


/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [07:13:14] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [26]:
y_pred_xgb = xgboost_model.predict(valid)

In [27]:
y_pred_xgb[:10]

array([10.563532, 23.15104 , 15.480803, 16.927462, 17.899588, 12.403639,
       17.757383, 10.913446, 11.362459, 12.970022], dtype=float32)

In [28]:
y_val[:10]

array([ 1.68333333, 32.08333333, 13.3       , 14.63333333, 27.95      ,
        3.73333333, 13.26666667,  3.71666667,  5.25      ,  7.35      ])